In [ ]:
# Requires brainvisa

In [1]:
import anatomist.api as anatomist

In [34]:
from soma import aims
import json
from soma.qt_gui.qt_backend import Qt
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np

In [3]:
a = anatomist.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module anacontrolmenu
loading module foldsplit
loading module simple_controls
loading module histogram
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [4]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

result_file = "/neurospin/dico/agaudin/Runs/09_new_repo/Output/grid_searches/schiz_stats_by_region.csv"

In [5]:
with open(json_regions) as f:
    regions = json.load(f)

In [6]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [41]:
res = pd.read_csv(result_file, index_col=0)
#res = res[["Region (sulci_region_gridsearch)", "AUC", "std_auc"]].rename(columns = {"full_name": "region"})
res = res.rename(columns = {"full_name": "region"})
res.loc['Sor', 'region'] = "S.Or."
res.loc['Sor_olfactif', 'region'] = "S.Or.-S.Olf."
res

,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final
cingulate,CINGULATE.,0.532325,0.551791,0.006604,0.005809,1.210309e+01,5.399155,0.585110,0.543461,0.008360,0.019361,1.355780,3.581346
BROCA,S.F.inf.-BROCA-S.Pe.C.inf.,0.509573,0.515282,0.004978,0.003315,4.957421e+00,1.527788,0.500891,0.539656,0.011979,0.011790,1.850623,0.325930
pericalcarine,Sc.Cal.-S.Li.,0.525666,0.531585,0.004609,0.003554,1.205652e+01,6.479759,0.557747,0.547025,0.013569,0.020130,1.399449,2.183796
STs,S.T.s.,0.522412,0.553065,0.006333,0.004731,1.562726e+01,3.394259,0.537043,0.572788,0.020517,0.011215,2.837745,1.138738
STs_br,S.T.s.br.,0.513011,0.540242,0.004988,0.003440,1.658451e+01,2.240244,0.539354,0.524826,0.003447,0.012663,1.216549,3.774895
SC_sylv,S.C.-sylv.,0.521498,0.536435,0.005174,0.003722,1.354546e+01,4.274225,0.528843,0.548716,0.007613,0.022716,1.307228,2.015737
precentral,S.C.-S.Pe.C.,0.526962,0.542408,0.005295,0.003888,1.534828e+01,5.709904,0.542782,0.508260,0.004516,0.010755,0.615044,3.460453
postcentral,S.C.-S.Po.C.,0.510137,0.505425,0.006201,0.004280,9.789316e-01,1.269838,0.501102,0.500785,0.012495,0.022955,0.312315,0.330712
Sor,S.Or.,0.495816,0.461522,0.005502,0.003772,2.603684e-15,0.110689,0.458124,0.449381,0.015347,0.006541,0.000326,0.011556
Sor_olfactif,S.Or.-S.Olf.,0.490450,0.477443,0.005483,0.005369,1.974030e-05,0.019265,0.496074,0.453519,0.021415,0.014674,0.007432,0.245441


In [43]:
res = res.drop(index="fronto-parietal_medial_face")

In [44]:
def get_sulci(region):
    list_sulci = list(regions['brain'][f"{region}_right"].keys())
    list_sulci = ["S.F.int._right" if x=="paracingular._right" else x for x in list_sulci]
    return list_sulci

In [45]:
get_sulci("CINGULATE.")

['S.F.int._right', 'F.C.M.ant._right']

In [46]:
res

,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final
cingulate,CINGULATE.,0.532325,0.551791,0.006604,0.005809,1.210309e+01,5.399155,0.585110,0.543461,0.008360,0.019361,1.355780,3.581346
BROCA,S.F.inf.-BROCA-S.Pe.C.inf.,0.509573,0.515282,0.004978,0.003315,4.957421e+00,1.527788,0.500891,0.539656,0.011979,0.011790,1.850623,0.325930
pericalcarine,Sc.Cal.-S.Li.,0.525666,0.531585,0.004609,0.003554,1.205652e+01,6.479759,0.557747,0.547025,0.013569,0.020130,1.399449,2.183796
STs,S.T.s.,0.522412,0.553065,0.006333,0.004731,1.562726e+01,3.394259,0.537043,0.572788,0.020517,0.011215,2.837745,1.138738
STs_br,S.T.s.br.,0.513011,0.540242,0.004988,0.003440,1.658451e+01,2.240244,0.539354,0.524826,0.003447,0.012663,1.216549,3.774895
SC_sylv,S.C.-sylv.,0.521498,0.536435,0.005174,0.003722,1.354546e+01,4.274225,0.528843,0.548716,0.007613,0.022716,1.307228,2.015737
precentral,S.C.-S.Pe.C.,0.526962,0.542408,0.005295,0.003888,1.534828e+01,5.709904,0.542782,0.508260,0.004516,0.010755,0.615044,3.460453
postcentral,S.C.-S.Po.C.,0.510137,0.505425,0.006201,0.004280,9.789316e-01,1.269838,0.501102,0.500785,0.012495,0.022955,0.312315,0.330712
Sor,S.Or.,0.495816,0.461522,0.005502,0.003772,2.603684e-15,0.110689,0.458124,0.449381,0.015347,0.006541,0.000326,0.011556
Sor_olfactif,S.Or.-S.Olf.,0.490450,0.477443,0.005483,0.005369,1.974030e-05,0.019265,0.496074,0.453519,0.021415,0.014674,0.007432,0.245441


In [47]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final,sulcus
cingulate,CINGULATE.,0.532325,0.551791,0.006604,0.005809,12.103088,5.399155,0.585110,0.543461,0.008360,0.019361,1.355780,3.581346,"[S.F.int._right, F.C.M.ant._right]"
BROCA,S.F.inf.-BROCA-S.Pe.C.inf.,0.509573,0.515282,0.004978,0.003315,4.957421,1.527788,0.500891,0.539656,0.011979,0.011790,1.850623,0.325930,"[S.F.inf._right, S.F.inf.ant._right, F.C.L.r.a..."
pericalcarine,Sc.Cal.-S.Li.,0.525666,0.531585,0.004609,0.003554,12.056517,6.479759,0.557747,0.547025,0.013569,0.020130,1.399449,2.183796,"[F.Cal.ant.-Sc.Cal._right, S.O.p._right, S.Li...."
STs,S.T.s.,0.522412,0.553065,0.006333,0.004731,15.627258,3.394259,0.537043,0.572788,0.020517,0.011215,2.837745,1.138738,[S.T.s._right]
STs_br,S.T.s.br.,0.513011,0.540242,0.004988,0.003440,16.584515,2.240244,0.539354,0.524826,0.003447,0.012663,1.216549,3.774895,"[F.I.P.r.int.1_right, F.I.P.r.int.2_right, S.T..."


In [48]:
res = res.sort_values(by="best_test_auc", ascending=True)
res = res.explode("sulcus")
res.head()

,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final,sulcus
Sor,S.Or.,0.495816,0.461522,0.005502,0.003772,2.603684e-15,0.110689,0.458124,0.449381,0.015347,0.006541,0.000326,0.011556,S.Or._right
Sor_olfactif,S.Or.-S.Olf.,0.490450,0.477443,0.005483,0.005369,1.974030e-05,0.019265,0.496074,0.453519,0.021415,0.014674,0.007432,0.245441,S.Or._right
Sor_olfactif,S.Or.-S.Olf.,0.490450,0.477443,0.005483,0.005369,1.974030e-05,0.019265,0.496074,0.453519,0.021415,0.014674,0.007432,0.245441,S.Olf._right
FCMpost,F.C.M.post.-S.p.C.,0.524086,0.500535,0.004715,0.005186,3.380849e-01,5.736718,0.490245,0.542162,0.005707,0.020649,1.256872,0.036816,F.C.M.post._right
FCMpost,F.C.M.post.-S.p.C.,0.524086,0.500535,0.004715,0.005186,3.380849e-01,5.736718,0.490245,0.542162,0.005707,0.020649,1.256872,0.036816,S.C.LPC._right


In [49]:
for _, row in res.iterrows():
    print(row.sulcus)

S.Or._right
S.Or._right
S.Olf._right
F.C.M.post._right
S.C.LPC._right
S.p.C._right
S.F.int._right
S.R.inf._right
S.R.sup._right
S.s.P._right
S.Pa.int._right
S.C._right
S.C.sylvian._right
S.Po.C.sup._right
F.I.P.Po.C.inf._right
F.C.L.r.retroC.tr._right
S.F.marginal._right
S.F.orbitaire._right
S.F.inf.ant._right
S.F.inf._right
S.F.inf.ant._right
F.C.L.r.ant._right
F.C.L.r.asc._right
F.C.L.r.diag._right
S.Pe.C.inf._right
F.I.P._right
S.Po.C.sup._right
S.Pa.sup._right
F.I.P._right
F.I.P.Po.C.inf._right
S.F.median._right
S.F.polaire.tr._right
S.F.sup._right
F.C.L.p._right
F.C.L.r.sc.ant._right
F.C.L.r.sc.post._right
INSULA_right
F.C.L.a._right
F.P.O._right
S.Pa.t._right
S.Cu._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.F.inter._right
S.F.sup._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.Li.ant._right
S.Li.post._right
S.T.i.ant._right
S.T.i.post._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
S.C._right
S.C.sylvian._right
F.I.P.r.int.1_right
F.I

In [50]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,test_auc,best_test_auc,test_auc_se,best_test_auc_se,-log_p_value_best,-log_p_value_final,top5_test_auc,top5_best_test_auc,top5_test_auc_se,top5_best_test_auc_se,top5_-log_p_value_best,top5_-log_p_value_final,sulcus
SFmarginal,S.F.marginal-S.F.inf.ant.,0.508136,0.509374,0.004883,0.004235,1.813174,1.296782,0.499607,0.528934,0.009392,0.014437,1.238216,0.287627,S.F.orbitaire._right


In [51]:
g = aims.read(Rspam_model)
ag = a.toAObject(g)



for vertex in g.vertices():
    vertex['AUC'] = 0.

for _, row in res.iterrows():
    for vertex in g.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['best_test_auc']):
                vertex['AUC'] = row.best_test_auc-row.best_test_auc_se

ag.setColorMode(ag.PropertyMap)
ag.setColorProperty('AUC')
ag.notifyObservers()

w = a.createWindow('3D')
w.addObjects(ag)


In [53]:
res.columns

Index(['region', 'test_auc', 'best_test_auc', 'test_auc_se',
       'best_test_auc_se', '-log_p_value_best', '-log_p_value_final',
       'top5_test_auc', 'top5_best_test_auc', 'top5_test_auc_se',
       'top5_best_test_auc_se', 'top5_-log_p_value_best',
       'top5_-log_p_value_final', 'sulcus'],
      dtype='object')

In [54]:
res = res.sort_values(by="-log_p_value_best", ascending=True)

In [55]:
g2 = aims.read(Rspam_model)
ag2 = a.toAObject(g2)

for vertex in g2.vertices():
    vertex['-log10p'] = 0.

for _, row in res.iterrows():
    for vertex in g2.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['-log_p_value_best']):
                vertex['-log10p'] = row['-log_p_value_best']

ag2.setColorMode(ag2.PropertyMap)
ag2.setColorProperty('-log10p')
ag2.notifyObservers()

w2 = a.createWindow('3D')
w2.addObjects(ag2)


In [56]:
res = res.sort_values(by="top5_-log_p_value_best", ascending=True)

In [57]:
g2 = aims.read(Rspam_model)
ag2 = a.toAObject(g2)

for vertex in g2.vertices():
    vertex['-log10p'] = 0.

for _, row in res.iterrows():
    for vertex in g2.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['top5_-log_p_value_best']):
                vertex['-log10p'] = row['top5_-log_p_value_best']

ag2.setColorMode(ag2.PropertyMap)
ag2.setColorProperty('-log10p')
ag2.notifyObservers()

w2 = a.createWindow('3D')
w2.addObjects(ag2)


In [58]:
res = res.sort_values(by="top5_best_test_auc", ascending=True)

In [60]:
g2 = aims.read(Rspam_model)
ag2 = a.toAObject(g2)

for vertex in g2.vertices():
    vertex['AUC'] = 0.

for _, row in res.iterrows():
    for vertex in g2.vertices():
        vname = vertex.get('name')
        if vname == row.sulcus:
            if pd.notnull(row['top5_best_test_auc']):
                vertex['AUC'] = row['top5_best_test_auc'] -row.top5_best_test_auc_se

ag2.setColorMode(ag2.PropertyMap)
ag2.setColorProperty('AUC')
ag2.notifyObservers()

w2 = a.createWindow('3D')
w2.addObjects(ag2)
